In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import scipy.interpolate as spi
import scipy.optimize as spo
import scipy.stats as sps
import git
import altair as alt
import hysttools as ht
from pathlib import Path
from datetime import date

In [ ]:
from altair_data_server import data_server
alt.data_transformers.enable('data_server')

In [ ]:
repo=git.Repo('.', search_parent_directories=True)
path_root = Path(repo.working_tree_dir)
data_folder = path_root / 'data'
processed_folder = data_folder / 'processed'
vsm_folder = data_folder / 'raw'/'vsm'
image_folder = data_folder / 'images'
search_pattern = re.compile('.*irmdcd$') # This is the suffix for a hysteresis file

file_list = list(filter(search_pattern.match,os.listdir(vsm_folder)))
specimens=pd.DataFrame(data={'name':file_list,'file':file_list})
specimens['name']=specimens['name'].str.replace('\.irmdcd','').str.replace('-.*','').str.lower()

location_names={'.*eg.*':'glacier','.*cb.*':'bay','.*combay.*':'bay','.*suspended.*':'suspended','.*bed.*':'bed'}
specimens['location']=specimens['name'].replace(location_names.keys(),location_names.values(),regex=True)

print(specimens)
print("\n{} IRMDCD files to process.".format(np.shape(specimens)[0]))

In [ ]:
#Hgrid=ht.make_hgrid(200,0.8,0.5)
#Hgrid2=ht.make_hgrid2(200,0.8)
#hyst_rh=pd.DataFrame(columns=['id']+["{}".format(x) for x in Hgrid])
#hyst_ih=pd.DataFrame(columns=['id']+["{}".format(x) for x in Hgrid])
s_dcd=pd.DataFrame([])
d_dcd=pd.DataFrame([])

In [ ]:
ht.process_irm??

In [ ]:
file_skip=[]
hyst_stats=pd.DataFrame(columns=['id','field','remanence','gradient','Hcr','max_d2'])
all_irm=pd.DataFrame(columns=['id','field','remanence','gradient','Hcr','max_d2'])

for index,row in specimens.iterrows():
    name,file_name,location = row
    file_path = vsm_folder / file_name
    print('Working on {}\n{}\n'.format(file_name,file_path))
    if (file_name in file_skip):
        print('Skipping... {}\n'.format(name))
    if (file_name not in file_skip):
        print("Processing: {}\n".format(name))
        s,d = ht.read_dcd(file_path)
        print("IRM/Backfield file read with {} data points.".format(d.shape[0]))
        #s_dcd=pd.concat([s,s_dcd],axis=0,ignore_index=True)
        #d_dcd=pd.concat([d,d_dcd],axis=0,ignore_index=True)
        result_dcd = ht.process_dcd(s,d)
        print("Backfield processed.")
        irm_curve=d.loc[(d['type']=='irm')].copy()
        smooth = 5*(10**(2.0*np.ceil(np.log10(0.1*np.min(np.abs(irm_curve.remanence))))))
        result_irm = ht.process_irm(s,d,smooth=smooth)
        to_add=pd.Series(result_irm['irm_derivative'].to_dict())
        to_add['id']=name
        max_d2=np.max(np.gradient(pd.Series(to_add['gradient']).values,pd.Series(to_add['field']).values))
        print(">> Max IRM second derivative: {}".format(max_d2))
        to_add['max_d2']=max_d2
        to_add['Hcr']=float(result_dcd['Hcr'])
        hyst_stats=hyst_stats.append(to_add,ignore_index=True)
        #if (max_d2>=2000):
        #    print("WARNING: High second derivative detected in IRM acquisition curve for specimen {}.".format(to_add['id'].first()))
        all_irm=all_irm.append(to_add,ignore_index=True)
        print("IRM data processed.")
    

In [ ]:
print(all_irm.columns)
#filtered_irm=all_irm[all_irm.apply(lambda x: (pd.Series(x['gradient'])>20).any(),axis=1)]
grouped_irm=all_irm.groupby('id')
irm_max_d2s=grouped_irm['max_d2'].first()
with pd.option_context('display.max_rows', 150):
    display(irm_max_d2s.sort_values())
filtered_irm=all_irm[all_irm.apply(lambda x: (pd.Series(x['max_d2'])<100000).any(),axis=1)]
#filtered_irm=
print(filtered_irm.shape,all_irm.shape)

In [ ]:
all_irm

In [ ]:
#filtered_irm=all_irm[all_irm.apply(lambda x: (pd.Series(x['gradient']).apply(lambda y: np.abs(y))>=20).any(),axis=1)].copy().reset_index(drop=True)
filtered_irm=all_irm[all_irm.apply(lambda x: (pd.Series(x['max_d2'])<2000).any(),axis=1)].copy().reset_index(drop=True)
#grouped_filtered_irm=filtered_irm.groupby('id')
#display(pd.concat(pd.DataFrame({'id':id, 'field':field,'remanence':remanence,'gradient':gradient,'max_d2':max_d2}) 
#                  for id, field, remanence, gradient, max_d2 in filtered_irm.items()))
filtered_irm=pd.merge(filtered_irm,specimens,left_on='id',right_on='name')
df_all=pd.DataFrame([])
for index,row in filtered_irm.iterrows():
    df=pd.DataFrame({'id':row['id'],'location':row['location'],'field':pd.Series(row['field']),'remanence':pd.Series(row['remanence']),
                    'gradient':pd.Series(row['gradient']),'max_d2':row['max_d2']})
    df_all=pd.concat([df_all,df],axis=0)

#print (df_all)

P=alt.Chart(df_all).mark_line(opacity=0.5).encode(
    x=alt.X('field:Q',scale=alt.Scale(type='log',domain=(0.01,1),clamp=True)),
    y=alt.Y('gradient:Q',scale=alt.Scale(domain=(0,20),clamp=True)),
    detail='id:N',
#    color='',
    tooltip=['id','field'],
    facet='location:N'
).properties(
    width=300,
    height=300
).interactive()

display(P)